<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("compléments", "asyncio")

# `asyncio`

### objectifs

* faciliter l'écriture "de code parallèle de manière séquentielle"
* parallèle: on fait **plusieurs choses** en même temps
* séquentielle: ça se passe dans **un seul thread**
* défini dans [pep3156](https://www.python.org/dev/peps/pep-3156/)

## historique

* défini comme une convergence 
* de différentes approches similaires
* développées dans les frameworks web (tornado notamment)
* dispo sous cette forme depuis python-3.5
* et aussi en 3.4 avec une syntaxe différente

# un exemple

* récupérer plusieurs urls en parallèle

In [ ]:
import asyncio
import aiohttp
 
async def asynchroneous(url):
    async with aiohttp.ClientSession() as session:
        print(f"fetching {url}")
        async with session.get(url) as response:
            print(f"{url} returned status {response.status}")
            raw = await response.read()
            print(f"{url} returned {len(raw)} bytes")

In [ ]:
async def main(urls):
    """
    Creates a group of coroutines and waits for them to finish
    """
    return await asyncio.gather(* (asynchroneous(url) for url in urls))

In [ ]:
urls = ["http://www.irs.gov/pub/irs-pdf/f1040.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf"]

In [ ]:
import time
begin = time.time()
event_loop = asyncio.get_event_loop()
event_loop.run_until_complete(main(urls))
print("Durée totale {}s".format(time.time() - begin))

## pour comparer

In [ ]:
# en version purement séquentielle
import requests

def synchroneous(url):
    print(f"fetching {url} synchroneously (blocking)")
    response = requests.get(url)
    print(f"{url} returned status {response.status_code}")
    print(f"{url} returned {len(response.text)} chars")

In [ ]:
import time
begin = time.time()
for url in urls:
    synchroneous(url)
print("Durée totale {}s".format(time.time() - begin))

# bien remarquer

* dans notre exemple nous n'avons défini aucune callback
* ni aucun *thread*
* le code de la version `asyncio` 
  * a un flux de contrôle 'normal'
  * même si on découpe un peu plus finement l'algorithme
* on pourrait faire tourner en parallèle n'importe quelle tâche réactive
  * comme réagir à une entrée clavier
  * faire tourner un processus séparé
  * lire un fichier local...

# les morceaux

##### syntaxe:

* `async def` pour définir une fonction asynchrone
* `await <expr>` pour attendre un résultat asynchrone
* `await for` pour un itérateur qui attend à chaque tour
* `await with` ditto pour un context manager

##### librairie `asyncio`

* une boucle d'événement, par exemple `asyncio.get_event_loop()`
* le coeur de la librairie est collé à l'OS et tire parti du framework

# pourquoi c'est mieux que des threads

* l'instruction `await`
  * pour indiquer les points où on peut changer de contexte
* contrairement à ce qui se passe en multi-thread
  * où on passe d'un thread à l'autre de manière non controlée
* pas (beaucoup moins) de **section critiques**
  * moins de charge sur le programmeur
* globalement plus efficace

# un autre exemple

In [ ]:
import asyncio
# quelque chose de plus basique

async def mysleep(duration):
    print("Entrée dans {duration}".format(**locals()))
    await asyncio.sleep(duration)
    print("Sortie de {duration}".format(**locals()))

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(
    asyncio.gather(mysleep(1), mysleep(0.5), mysleep(1.5)))

# ce qu'il faut retenir

* quasi totalité des librairies réseau disponibles
  * http, telnet, ssh, ...
* et pour subprocess (natif dans `asyncio`)
* préférez cette solution 
  * dès que vous devez faire quelque chose de réactif
  * de préférence aux threads

# comment ça marche

* à l'origine il y a les fonctions génératrices

In [ ]:
def gen_range(n):
    i = 0
    while i <= n:
        yield i
        i += 1

In [ ]:
def enumerer():
    for x in gen_range(2):
        print(x)

enumerer()

# fonctions génératrices 

* déjà à ce stade il se passe quelque chose de *magique*
* ou en tous cas de pas simple à implémenter avec un pc et une pile
* python 'séquentiel' a déjà un mécanisme 
  * pour mettre au freezer une fonction et son contexte

# coroutine

* une coroutine (`async def`) est une généralisation de fonction génératrice
* historiquement `await` s'appelait `yield from`